# Chapter 6

Outros métodos de gerar modelos lineares sem usar mínimos quadrados que podem trazer melhor acurácia e interpretabilidade dependendo do cenário (sem perder a rigidez de um modelo linear).

## Subset Selection

Identifica um subconjunto de preditores que estão relacionados à resposta e treina o modelo (usando mínimos quadrados).

Esse treino é feito de forma exaustiva, ou seja, todas as combinações possíveis de variáveis são treinadas até achar a com melhor resultado.

### Best Subset Selection

```
M0 = modelo nulo (sem preditores)
Para k em 1,2,... p:
    Ajuste todos os modelos que contém exatamente k preditores
    Escolha o com menor RSS ou maior R² e chame de Mk
Escolha o melhor modelo de entre M0,... Mp usando o erro predito na validação cruzada, AIC, BIC ou R² ajustado.
```

problemas estatísticos e baixa performance estão associados com grandes valores de p.   

### Forward Stepwise Selection

O método de forward stepwise selection é computacionalmentem muito maias eficiente por passar por menos combinações, porém pode não convergir para o ponto crítico:

```
M0 = modelo nulo (sem preditores)
Para k em 0,1,...,p-1:
    considere todos os modelos que aumentam os preditores em Mk com um preditor adicional
    Escolha o melhor desses modelos e chame de Mk+1 com base em alguma métrica
Escolha o melhor modelo com base em alguma métrica de teste.
```

### Backward Stepwise Selection

Equivalente ao backward mas ao invés de partir dos modelos com menos preditores para os com mais preditores, começa dos modelos com mais preditores. Se souber de antemão que o resultado final envolverá muitas variáveis, pode ser mais rápido, mas não funciona quando existem menos exemplos que preditores.

```
Mp = modelo completo (com todos os preditores)
Para k em p,p-1,...,1:
    Considere todos os modelos sem um dos preditores em Mk
    Escolha o melhor desses modelos e chame de Mk-1 com base em alguma métrica
Escolha o melhor modelo com base em alguma métrica de teste
```

**OBS: Existem modelos híbridos que adicionam ou subtraem dependendo do contexto, que podem ser situacionais mas eficientes em certas situações.**

### Estimando o melhor modelo

RSS e $R^2$ sempre ficam menores conforme o número de variáveis aumenta então não são nesse contexto boas métricas para definir o melhor modelo no subset selection. Existem algumas métricas que funcionam bem com muitas variáveis porque punem modelos com muitas variáveis que não reduzem significativamente o erro de teste estimado:

#### $C_p$ de Mallow

Adiciona ao RSS uma penalidade para ajustar a diferença entre erros de treino e teste.

$$
    C_p = \frac 1 n (RSS + 2d\hat\sigma^2) \\
    \hat\sigma^2: \text{Variância do erro associado com o dataset de teste}
$$

geralmente o modelo escolhido para obter a resposta com que o erro é calculada é o modelo com todos os preditores. É possível provar que se $\hat \sigma^2$ não for uma estimativa enviesada, $C_p$ também é uma estimativa não enviesada do MSE de teste.

#### Akaike Information Criterion (AIC)

Funciona para modelos que podem ser ajustados por máxima verossimilhança. No caso de modelos com erro gaussiano, como a mínimos quadrados e verossimilhança são o mesmo, o AIC é proporcional a

$$
    AIC = \frac 1 n (RSS + 2d\hat{\sigma}^2)
$$

**OBS: algumas constantes foram omitidas.**

No caso de modelos que podem ser otimizados por mínimos quadrados, as duas métricas são equivalentes. Mas o AIC continua sendo válido para predições que usam máxima verossimilhança.

#### Bayesian Information Criterion (BIC)

Parte de um ponto de vista bayesiano mas possui resultados similares a BIC. Para mínimios quadrados e $d$ preditores é:

$$
    BIC = \frac 1 n (RSS + \log(n) d \hat \sigma^2)
$$

Por ser logarítmico possui penalidade maior para modelos com muitas variáveis, geralmente resultando em modelos com menos variáveis.

#### $R^2$ Ajustado

É a métrica mais simples e com menos justificativa estatística para reduzir o viés dos modelos com melhores métricas.

$$
    AR^2 = 1 - \frac{RSS/(n - d - 1)}{TSS/(n - 1)}
$$

Intuitivamente, o $R^2$ ajustado cria uma noção de custo para cada variável que só é justificado se a variável explica a variância dos dados ao invés de gerar mais variância (ruído).

**DÚVIDA: O livro parece considerar que ou validação cruzada é feita ou as métricas acima são usadas. As duas coisas são mutuamente exclusivas?**

## Shrinkage

Ajusta um modelo envolvendo todos os preditores mas os coeficientes estimados são diminuídos tendendo a zero relativos às estimativas dos mínimos quadrados (processo chamado de regularização), fazendo com que alguns coeficientes possivelmente se tornem nulos, fazendo escolha de variáveis.

### Ridge

Ao invés de minimizar a regressão usando mínimos quadrados, que minimiza o RSS, o ridge minimiza a seguinte métrica:
$$
    \sum^n_{i=1}\left(y_i-\beta_0-\sum^p_{j=1}\beta_jx_{ij}\right) + \lambda \sum^p_{j=1}\beta^2_j=RSS + \lambda \sum^p_{j=1}\beta^2_j \\ ~ \\
    \lambda \geq 0
$$

Lambda é um parâmetro de ajuste (ou hiperparâmetro) que precisa ser determinado antes do treino e $\lambda \Sigma$  é chamado de penalidade de redução. Seu valor é pequeno quando $\Beta \to 0$, logo minimizar a penalidade também minimiza $\Beta$. $\lambda$ controla o efeito da penalidade na métrica, logo uma boa estimativa é importante.

Nota: Usando Ridge, $\beta_0$, o coeficiente linear, não é considerado porque representa o valor médio da resposta quando todos os parâmetros são nulos. Se o dataset for normalizado, $\beta_0$ será igual à média. Normalização dos dados afeta o resultado final do ridge e é uma boa prática.
$$
    \tilde x_{ij} = \frac {x_{ij}} {\sqrt{\frac 1 n \sum^n_{i=1}(x_{ij} - \bar x_j)^2}}
$$

**Normalização L2 ou $\|\beta\|_2$:** Mede a distância dos coeficientes de 0 e reduz conforme $\lambda$ aumenta.
$$
    \|\beta\|_2 = \sqrt{\sum^p_{j=1}\beta^2_j}
$$

Computacionalmente Ridge faz muito menos operações que best subset selection.

### Lasso

Técnica baseada em Ridge que faz seleção de preditores para evitar overfitting.
$$
    \sum^n_{i=1}\left(y_i-\beta_0-\sum^p_{j=1}\beta_jx_{ij}\right)^2 + \lambda \sum^p_{j=1}\|\beta_j\| = RSS + \lambda \sum^p_{j=1}\|\beta_j\|
$$
A penalidade em Lasso, usa a regularização L1 ao invés de L2, o que causa com que alguns coeficientes se tornem zero (no Ridge nenhum valor efetivamente é zerado), fazendo seleção de variáveis, gerando modelos muito mais simples de entender que aqueles produzidos por Ridge. 
$$
    \|\beta\|_1 = \sum \|\beta_j\|
$$

Como em Ridge os coeficientes nunca são zerados, é possível que os coeficientes obtidos sejam os mesmos que os obtidos usando mínimos quadrados. Já usando Lasso, é garantido que os resultados sejam diferentes.

**TODO**: Como otimizar $\lambda$?

## Dimension Reduction

Projeta todos os preditores em um espaço dimensional menor que o original usando combinações lineares (ou projeções), e então um modelo é treinado usando mínimos quadrados.